<a href="https://colab.research.google.com/github/nicolenair/capstone/blob/master/anno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import os
import re

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.listdir("/content/gdrive/My Drive/Nicole-Internship-Complete-Folder/chatbot - all/chatbot-flask/")

['templates',
 'static',
 '.DS_Store',
 'chat.py',
 'dec_supreme.txt',
 'enc_supreme.txt',
 'train.dec.txt',
 'test.enc.txt',
 'test.dec.txt',
 'train.enc.txt',
 'model-eng-dec.bin',
 'model-eng-enc.bin',
 'GoogleNews-vectors-negative300.bin.gz',
 'model-eng-dec_emb.bin',
 'model-eng-enc_emb.bin',
 'chat_emb.py']

In [0]:
books = ["gables", "avonlea", "island", "dreams"]
urls = ["https://www.gutenberg.org/files/45/45-0.txt", "https://www.gutenberg.org/files/47/47-0.txt", "https://www.gutenberg.org/files/51/51-0.txt", "http://www.gutenberg.org/cache/epub/544/pg544.txt"]

for i in range(len(books)):
    page = requests.get(urls[i])
    file = open("{}.txt".format(books[i]) , "w+")
    file.write(page.text)
    file.close()

file = open("gables.txt", "r")
gables = file.read()
file = open("avonlea.txt", "r")
avonlea = file.read()
file = open("island.txt", "r")
island = file.read()
file = open("dreams.txt", "r")
dreams = file.read()

In [0]:
#returns list of paragraphs in a book that can be innputted into train_classifier
def paragraph_list(book):
    current_total = 0
    chapter_lengths = []
    full_para_list = []
    lengths = [38, 0, 40, 0]
    if book==gables:
        n_chap = lengths[0]
    elif book == dreams:
        n_chap = lengths[2]
    for i in range(1, n_chap):
        if book == dreams:
            index_c1 = book[re.search("CHAPTER {}".format(i), book).span()[0] + 11: re.search("CHAPTER {}".format(i + 1), book).span()[0]]
        if book == gables:
            nums = ["I", "II","III","IV","V", "VI", "VII", "VIII", "IX", "X", "XI", "XII", "XIII","XIV", "XV", "XVI", "XVII", "XVIII", "XIX", "XX", "XXI", "XXII", "XXIII", "XXIV", "XXV", "XXVI","XXVII" ,"XXVIII","XXIX", "XXX", "XXXI", "XXXII", "XXXIII","XXXIV", "XXXV" ,"XXXVI", "XXXVII", "XXXVIII"]
            first1 = re.search("CHAPTER {}\W".format(nums[i-1]), gables).span()[0]
            first2 = re.search("CHAPTER {}\W".format(nums[i]), gables).span()[0]
            index_c1 = gables[re.search("CHAPTER {}\W".format(nums[i-1]), gables[first1 + 1:]).span()[0] + first1: re.search("CHAPTER {}\W".format(nums[i]), gables[first2 + 1:]).span()[0] + first2]
        index_c1 = re.sub("(\n)(\w)", " \g<2>", index_c1)
        
        chapter_lengths.append(len(index_c1) - current_total)
        split_para_c1 = [s.strip() for s in index_c1.splitlines() if len(s)>10]
        full_para_list.append(split_para_c1)
        current_total += len(index_c1)  
        
    return full_para_list

full_para_list = paragraph_list(dreams)

#data set, be very careful with cell!!!



def remove_names(full_para_list, target):
    for o, i in enumerate(full_para_list):
        for t, e in enumerate(i):
            for k in name_mapping.keys():
                if target[o][t]==name_mapping[k]:
                    full_para_list[o][t] = re.sub(k, "", full_para_list[o][t])
                    full_para_list[o][t] = re.sub(k.lower(), "", full_para_list[o][t])
    return full_para_list
#have a more comprehensive way names could appear
#also try to do the said thing, rather than just removing all names

# full_para_list = remove_names(full_para_list[:10], target_dreams)

In [0]:
full_para_list_gables = paragraph_list(gables)
full_para_list_dreams = paragraph_list(dreams)

In [0]:
def return_target(book):
  #name_mapping = {"narrator":0, "Anne":1, "Rachel":2, "Marilla":3, "Gilbert":4, "Captain Jim":5, "Diana":6, "Mrs. Harmon Andrews":7, "Mrs. Jasper":8, "Paul":9, "Charlotta the Fourth":10, "Philippa":11, "Miss Patty":12, "Mrs. Doctor":13, "Doctor Dave":14, "Cornelia":15, "Leslie":16, "unnamed char": 17, "Matthew":18, "Mrs. Spencer":19, "Mrs. Barry":20, "Mr Philips": 21, "Charlie Sloane":22, "Aunt Josephine":23, "Mrs Allan":24}
  if book == "gables":
    return [[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 0, 0, 2, 0, 3, 0, 2, 3, 0, 2, 0, 3, 3, 0, 2, 0, 3, 2, 3, 0, 2, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0, 17, 0, 17, 18, 17, 0, 0, 0, 0, 1, 0, 18, 1, 0, 0, 1, 18, 1, 18, 1, 18, 1, 0, 18, 1, 18, 1, 18, 1, 0, 18, 0, 1, 18, 1, 18, 1, 18, 1, 0, 0, 0, 18, 0, 1, 18, 1, 18, 1, 0, 18, 1, 0, 18, 1, 18, 1, 18, 1, 18, 1, 0, 18, 1, 0, 1, 0, 18, 1, 0,0, 1, 0], 
                 [0, 0, 3, 18, 0, 3, 18, 3, 0, 1, 0, 3, 1, 0, 3, 0, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 18, 18, 3, 1, 3, 0, 3, 1, 3, 1, 3, 1, 18, 0, 0, 3, 0, 1, 3, 0, 0, 3, 0, 1, 0, 0, 3, 18, 3, 18, 3, 0, 18, 3, 18, 3, 18, 18, 3, 18, 3, 18, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 3, 1, 3, 3, 0, 1, 3, 0, 0, 0, 0, 3, 1, 3, 1, 3, 3, 0, 0, 3, 1, 3, 1, 3, 1, 3, 0, 3, 0, 3, 0, 0, 3, 0], 
                 [0, 1, 3, 0, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 1, 1, 0, 3, 1, 3, 0, 3, 0, 1, 3, 1, 1], 
                 [0, 0, 19, 1, 3, 1, 19, 19, 17, 19, 3, 19, 0, 0, 19, 19, 0, 0, 19, 19, 0, 17, 17, 0, 3, 17, 0, 1, 3, 1, 0, 3, 1, 0, 0, 18, 3, 0, 18, 3, 18, 0, 3],
                 [0, 0, 3, 1, 3, 1, 0, 3, 1, 0, 3, 1, 3, 1, 0, 3, 1, 3, 0, 1, 0, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 3, 1, 0, 18],
           [0, 0, 0, 1, 3, 0, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 1, 3, 1, 3, 0, 3, 0, 1, 3, 0, 1, 3, 0, 1, 3, 0, 1, 3, 1, 3, 0, 1, 3, 0, 0, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 0, 1, 0, 1, 0, 1, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 2, 3, 0, 2, 3, 0, 2, 0, 0, 1, 2, 0, 1, 0, 2, 3, 0, 2, 0, 3, 2, 3, 0, 2, 0, 0, 0, 3, 0, 3, 0, 3, 1, 3, 1, 3, 1, 0, 3, 1, 3, 0, 1, 3, 0],
           [0, 0, 18, 3, 18, 3, 3, 0, 0, 0, 0, 18, 0, 1, 0, 0, 1, 18, 1, 18, 1, 18, 1, 0, 3, 1, 3, 0, 3, 1, 0, 0, 1, 0, 0, 0, 2, 1, 2, 0, 2, 0, 1, 3, 3, 1, 1, 3, 1, 3, 3, 0, 1, 0, 3, 1, 3],
           [0, 3, 1, 0, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 0, 1, 0, 3, 0, 0, 0, 0, 3, 1, 3, 0, 1, 3, 1, 3, 1, 3, 1, 1, 1, 1, 0],
           [0, 0, 3, 1, 3, 1, 0, 3, 1, 3, 0, 1, 3, 0, 1, 0, 20, 3, 1, 0, 20, 1, 6, 20, 0, 0, 1, 0, 6, 1, 0, 6, 1, 6, 1, 6, 1, 0, 6, 0, 3, 1, 3, 0, 18, 3, 1, 3],
           [0, 3, 0, 1, 3, 1, 3, 1, 3, 1, 3, 1, 0, 3, 1, 3, 0, 0, 1, 3, 1, 1, 0, 0, 1], 
           [0, 0, 3, 1, 3, 1, 3, 1, 3, 1, 3, 0, 3, 1, 0, 3, 1, 3, 0, 3, 0, 0, 18, 3, 18, 3, 18, 0, 3, 1, 3, 1, 0, 0, 1, 3, 1, 0, 3, 1, 3, 1, 0, 3, 0, 3, 0, 0, 3, 1, 0, 18, 3, 18, 3, 0, 0, 0, 3, 0, 3, 1, 0, 3, 0, 1, 3, 1, 0,  1, 0, 3],
           [0, 1, 6, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 3, 1, 3, 0, 6, 0, 1, 6, 1,0, 6, 1, 6, 1, 6, 6, 0, 1, 0, 0, 0, 0, 0, 4, 0, 0, 1, 0, 0, 0, 21, 4, 0, 21, 0, 0, 0, 0, 4, 6, 1, 0, 6, 1, 0, 0, 0, 0, 0, 0, 21, 0, 21, 1, 21, 0, 0, 0, 0, 6, 1, 6, 1, 6, 1, 6, 0, 3, 1, 3, 1, 0, 0, 3, 0, 2, 3, 2, 3, 2, 0, 0, 0, 3, 1, 0, 3, ],
           [0, 0, 0, 3, 3, 1, 3, 1, 3, 1, 3, 1, 3, 0, 1, 6, 1, 6, 1, 0, 0, 0, 1, 0, 6, 1, 0, 6, 1, 0, 6, 1, 6, 1, 6, 1, 6, 0, 0, 3, 0, 3, 0, 0, 0, 3, 1, 3, 0, 3, 1, 3, 1, 3, 1, 1, 3, 0, 1, 3, 0, 0, 20, 0, 1, 0, 20, 0, 1, 20, 0, 1, 3, 0, 3], 
           [0, 0, 1, 0, 6, 1, 6, 1, 6, 1, 6, 1, 6, 1, 1, 0, 1, 3, 0, 1, 3, 1, 0, 0, 0, 0, 0, 1, 0, 29, 29, 0, 6, 6, 6, 6, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1],
           [0, 0, 0, 0, 1, 18, 1, 18, 0, 1, 18, 1, 18, 1, 18, 0, 1, 18, 1, 18, 0, 1, 6, 0, 1, 1, 6, 1, 0, 0, 0, 0, 0, 0, 0, 1, 17, 17, 0, 1, 18, 0, 1, 3, 0, 3, 0, 1, 3, 0, 1, 1, 6, 6, 1, 3, 1],
           [0, 1, 3, 1, 3, 1, 3, 1, 3, 0, 1, 3, 1, 3, 1, 3, 1, 3, 0, 18, 3, 18, 3, 18, 3, 18, 18, 0, 3, 0, 1, 3, 1, 0, 0, 0, 0,0,  1, 6, 0, 0, 0, 6, 1, 6, 6, 6, 1, 0, 23, 0, 1, 6, 1, 6, 0, 20, 0, 2, 1, 2, 1, 2, 1, 2, 2, 0, 1, 6, 1, 6, 1, 0, 6, 1, 6, 0, 1, 23, 1, 23, 1, 23, 1, 0, 23, 1, 0, 23, 1, 23, 0, 23, 3, 18, 23, 0, 23, 1], 
           [0, 0, 1, 3, 0, 1, 0, 0, 1, 3, 0, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 0, 3, 1, 3, 1, 3, 1, 3, 1, 3, 0, 1, 3, 0, 3, 1],
           [0, 1, 3, 1, 1, 0, 0, 1, 0, 1, 3, 0, 3, 1, 3, 1, 3, 0, 1, 6, 1, 6, 1, 0, 1, 3, 0, 1, 3, 1, 3, 0, 3, 0, 0, 3, 24, 0, 0, 1, 3, 1, 3, 0, 0, 3, 0, 1, 0, 1, 24, 0, 24, 1, 24, 0, 1, 3, 1, 3, 1, 3], 
           [0, 3, 1, 3, 0, 0, 0, 3, 1, 0, 0, 1],
           [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 25, 1, 25, 0, 6, 1, 0, ]]
  elif book == "dreams":
    return [[0, 1, 0, 0, 6, 1, 0, 6, 0, 1, 6, 1, 6, 1, 6, 1, 6, 1, 6, 1, 6, 1, 6, 1, 6, 1, 6, 6, 1, 6, 0, 1, 6, 1], 
          [0, 0, 3, 2, 0, 2, 2, 3, 3, 2, 3, 0, 0, 7, 1, 7, 8, 0, 4, 1, 4 ,4, 1, 4, 4, 1, 4, 1, 4, 1, 4, 1, 4, 1], 
          [0, 2, 1, 3, 1, 3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 0, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 2, 3, 2, 0,  0, 1, 9, 0, 1, 9, 1, 0, 10, 1, 10, 1, 10,0, 11, 1, 11, 1,11, 1,11,  1, 11, 1, 11, 12,1,0, 1, 1,0, 4, 1, 4, 1, 4, 1, 4, 1, 0],
          [0, 0, 1, 0, 0, 6, 3, 0, 2, 3, 6, 2, 6, 2, 1, 1, 2, 0, 0, 0], 
          [0, 0, 0, 1, 4, 1, 4, 0, 1, 0, 1, 4, 1, 4, 1, 1, 4, 1, 4, 0, 0, 4 ],
          [0, 0, 13, 0, 1, 1, 1, 0, 1, 0, 4, 0, 5, 0, 5, 0, 0, 0, 0, 5, 5, 14, 5, 13, 5, 13, 5, 13, 5, 13, 5, 1, 5, 0, 5, 5, 5, 14, 5, 5, 0, 4, 0, 5, 14, 4, 5, 13, 5],
          [0, 1, 4, 5, 1, 5, 5, 0, 5, 5, 5, 5, 5, 4, 14, 5, 5, 14, 5, 5, 5, 5, 5, 14, 1, 5, 5, 5, 5, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 1, 14, 5, 5, 1, 5, 5, 1, 5, 0, 1, 5, 0, 5, 1, 5, 1, 1, 5, 1, 5, 1, 1, 5, 5, 0, 5, 5, 13, 5, 0, 1],
          [0, 0, 1, 4, 1, 0, 1, 4, 0, 1, 4, 1, 4, 4, 0, 1, 4, 1, 4, 1, 0, 0, 15, 0, 15, 0, 15, 1, 15, 1, 15, 1, 15, 1, 15, 1, 15, 1, 15, 1, 15, 1, 15, 1, 15, 1, 0, 15, 0, 1, 15, 1, 15, 1, 15, 15, 15, 1, 0, 15, 1, 15, 1, 15, 1, 15, 1, 15, 1, 15, 15, 0, 15, 1, 15, 1, 15, 1, 15, 0, 1, 15, 1, 15], 
          [0, 0, 5, 0, 5, 5, 0, 5, 0, 1, 4, 1, 0, 1, 0, 4, 1, 4, 1, 0, 0, 5, 0, 5, 0, 5, 0, 5, 1, 0, 5, 4, 0, 5, 1, 5, 1, 5, 0, 5, 4, 5, 1, 5, 4, 0, 5, 0, 0, 0, 0, 0, 5, 5, 1, 1, 1, 0, 5, 0, 4, 1, 4, 1, 0], 
           [0, 1, 0, 1, 1, 0, 0, 1, 16, 0, 1, 16, 0, 1, 16, 1, 1, 16, 1, 16, 0, 16, 1, 16, 1, 16, 1, 16, 1, 16, 1, 16, 1, 16, 0, 1, 16, 1, 16, 1, 16, 0, 16, 1, 16, 1, 16, 1, 16, 0, 1, 16, 1, 0, 16, 0, 1, 16, 1, 16, 0, 4, 1, 4, 1, 4, 4, 1]]
  


In [512]:
full_para_list_gables[22][17]
                        

'If Anne had tumbled off the roof on the side up which she had ascended Diana would probably have fallen heir to the pearl bead ring then and there. Fortunately she fell on the other side, where the roof extended down over the porch so nearly to the ground that a fall therefrom was a much less serious thing. Nevertheless, when Diana and the other girls had rushed frantically around the house--except Ruby Gillis, who remained as if rooted to the ground and went into hysterics--they found Anne lying all white and limp among the wreck and ruin of the Virginia creeper.'

In [0]:
len(full_para_list_gables)

37